<a href="https://colab.research.google.com/github/ccskying/PythonNotes/blob/master/BigDataHW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf-8 -*-
"""
BigData homework1: perceptron algorithm and KNN algorithm
Chencheng Lu 2020/2/23
"""

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Data preparing

# Import tensorflow's MNIST data handle
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
#load data and preprocess data
mnist = tf.keras.datasets.mnist
(train_images, train_labels),(test_images, test_labels)=mnist.load_data()
#choose pictures of number 3 and number 5
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()#include all numbers from 0 to 9
index_train=np.where((train_labels==3)| (train_labels==5))#index of numbers 3 and 5 in training data
index_test=np.where((test_labels==3)|(test_labels==5))#index of numbers 3 and 5 in test data
train_images_35=train_images[index_train]
train_images_35=train_images_35.reshape((len(train_images_35),train_images_35[1].size))
#label of number 3: -1; label of number 5: +1
train_labels_35=train_labels[index_train].astype('int')
test_images_35=test_images[index_test]
test_images_35=test_images_35.reshape(len(test_images_35),train_images_35[1].size)
test_labels_35=test_labels[index_test].astype('int')
#change labels from '3 ' and '5’to '-1' and '+1'
train_labels_35[np.where(train_labels_35==3)]= -1
train_labels_35[np.where(train_labels_35==5)]=1
test_labels_35[np.where(test_labels_35==3)]= -1
test_labels_35[np.where(test_labels_35==5)]=1
#show the first 25 training data
# plt.figure(figsize=(10,10))
# for i in range(25):
#  plt.subplot(5,5,i+1)
#  plt.xticks([])
#  plt.yticks([])
#  plt.imshow(train_images_35[i].reshape((28,28)),)
#  plt.xlabel('number'+str(train_labels_35[i]))
# plt.show()
#append dummy feature 1 to feature vectors, and then normalize
train_images_35_w_dummy=np.insert(train_images_35,784,1,axis=1)/255
test_images_35_w_dummy=np.insert(test_images_35,784,1,axis=1)/255
#check the dimension, the feature vector of each sample shall be 785
print(train_images_35_w_dummy.shape[1])
print(test_images_35_w_dummy.shape)

# Data preparing end here

# Perceptron algorithm

# Define constants
weight = np.ones(785) # Step size of perceptron algrorithm
StepSize = 0.1
EndCondition = 0.80 # Perceptron algorithm training goal of accuracy
PA_enable = True # If true, activate perceptron algorithm, else, activate KNN algorithm

# Perceptron training: randomly pick data and train one time
def PerceptronTraining(weight):
  nPick = np.random.randint(0,train_images_35_w_dummy.shape[0])
  trainExample = train_images_35_w_dummy[nPick]
  fx = np.dot(trainExample,weight)
  if fx >= 0:
    y = 1
  else:
    y = -1
  # Test y with label, change w if y isn't fix the label
  if y != train_labels_35[nPick]:
    weight = weight + StepSize * train_labels_35[nPick] * train_images_35_w_dummy[nPick]
  return weight

# Return the accruracy of classifier
def CheckAccuracy(weight,testDataset,testLabel):
  fx = np.dot(testDataset,weight)
  fx[np.where(fx >= 0)] = 1
  fx[np.where(fx < 0)] = -1
  nAccuracy = np.sum(fx == testLabel) / testDataset.shape[0]
  return nAccuracy

# Loop training(perceptron classifier)
if PA_enable == True :
    nTrainTime = 0
    Accuracy = 0
    while Accuracy < EndCondition:
      weight = PerceptronTraining(weight)
      Accuracy = CheckAccuracy(weight,train_images_35_w_dummy, train_labels_35)
      nTrainTime += 1
      print("train ",nTrainTime," times, ","Accuracy: ", Accuracy)
    print("end at:",Accuracy)
    # After training, use test dataset to test the result.
    Accuracy = CheckAccuracy(weight, test_images_35_w_dummy, test_labels_35)
    print("test accuracy: ",Accuracy)

# Perceptron algorithm end here


# KNN classifier

# Define constants
nK = 1 # Pick how many neibghbers
isTrainingSamples = False # Using training samples or test samples to match training samples
# Calculate Euclidean distance 
def K_nearest_elements(sample):
    distance = np.sqrt(np.sum((train_images_35_w_dummy - sample)**2, axis = 1))
    return distance

# find k neighbers, classify the sample
def KNN_classifier(index,isTrainData):
    # Select Data and get the distance
    if isTrainData:
        sample = train_images_35_w_dummy[index]
    else:
        sample = test_images_35_w_dummy[index]
    distance = K_nearest_elements(sample)
    # sort the distance and get nearest neigbers
    # if training dataset, ignore itself
    KNNarrary = np.argsort(distance)
    if isTrainData:
        knn = KNNarrary[1:nK+1]
    else:
        knn = KNNarrary[0:nK]
    y = ClassifyLabel(KNNarrary,isTrainData)
    print(index,y, train_labels_35[knn],distance[knn])
    return y

# calculate y through k neighbers
def ClassifyLabel(KNNarrary,isTrainData):
    if isTrainData:
        knn = KNNarrary[1:nK+1]
    else:
        knn = KNNarrary[0:nK]
#    print(train_labels_35[knn])
    nNegative = np.sum(train_labels_35[knn] == -1)
    nPositive = np.sum(train_labels_35[knn] == 1)
    if nPositive > nNegative:
        y = 1
    elif nPositive < nNegative:
        y = -1
    else:
        y = np.random.choice([-1,1])
    return y

# check the result is true or false
def CheckLabel(y,index,isTrainData):
    if isTrainData:
        if y == train_labels_35[index]:
            return True
        else:
            return False
    else:
        if y == test_labels_35[index]:
            return True
        else:
            return False
    

# Check the total accuracy
def CheckPerformance(isTrainData):
    nAccuracy = 0
    if isTrainData:
        counts = train_images_35_w_dummy.shape[0]
    else:
        counts = test_images_35_w_dummy.shape[0]
    for index in range(counts):
        y = KNN_classifier(index,isTrainData)
        if CheckLabel(y,index,isTrainData):
            nAccuracy += 1
    if  isTrainData:
        print("train Accuracy",nAccuracy / train_images_35_w_dummy.shape[0])
    else:
        print("test Accuracy",nAccuracy / test_images_35_w_dummy.shape[0])

# Get the result of KNN algorithm
if PA_enable == False:        
    CheckPerformance(isTrainingSamples)

# KNN classifier end here
